In [1]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata, rrankdata
from pymcdm import weights as mcdm_weights

In [2]:
from prettytable import from_csv
with open("pima_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+----------+-----------+--------+-------------+---------+-------------+-------------+-------------+--------------+--------------+
|                      | ACCURACY | PRECISION | RECALL |     AUC     | F1SCORE |   FP_RATE   |   FN_RATE   | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+----------+-----------+--------+-------------+---------+-------------+-------------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.82   |    0.82   |  0.82  | 0.864458405 |   0.81  | 0.067961165 | 0.411764706 | 0.932038835 | 0.181818182  | 6.279813872  |
|    DECISION TREE     |   0.7    |    0.71   |  0.7   | 0.672758424 |   0.7   | 0.242718447 | 0.411764706 | 0.757281553 | 0.298701299  | 10.31690717  |
| GAUSSIAN NAIVE BAYES |   0.82   |    0.82   |  0.82  | 0.885018085 |   0.82  | 0.097087379 | 0.333333333 | 0.902912621 | 0.902912621  | 6.055551679  |
|         SVM          |   0.48   |    0.46   |  0.48  | 0.726537217 |   0.47  | 0

In [3]:
x_matrix, alt_names = mcdm.load("pima_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weight = [0.10909090908971901,0.16363636363821488, 0.18181818182165294,0.12727272727352062,0.14545454545314052,0.0363636363625124,0.0727272727239339,0.0545454545446777,0.09090909091046283,0.018181818182165292]
weights=np.array(weight)
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [4]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


1.0 0.9077876226958926
4.0 0.7015952559405871
3.0 0.7814123466750394
6.0 0.35392805625811224
5.0 0.6623382490628669
2.0 0.8504131712837436
7.0 0.18631707123302396


In [5]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

1.0 0.0
4.0 0.2596436663470678
3.0 0.23663310164846152
6.0 0.7608767886870238
5.0 0.2658241937561817
2.0 0.06550388718853742
7.0 1.0


In [6]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

1.0 0.7515151515176501
4.0 -0.14242424242496138
2.0 0.6575757575747051
6.0 -0.5757575757554767
5.0 -0.15151515151507428
3.0 0.2848484848518016
7.0 -0.8242424242486447


In [7]:
from prettytable import PrettyTable
print("#########################PIMA DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################PIMA DATASET#######################
+----------------------+--------+---------------------+-------+---------------------+-----------+----------------------+
|        Model         | TOPSIS |        Value        | VIKOR |        VALUES       | PROMETHEE |        VALUES        |
+----------------------+--------+---------------------+-------+---------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  1.0   |  0.9077876226958926 |  1.0  |         0.0         |    1.0    |  0.7515151515176501  |
|    DECISION TREE     |  4.0   |  0.7015952559405871 |  4.0  |  0.2596436663470678 |    4.0    | -0.14242424242496138 |
| GAUSSIAN NAIVE BAYES |  3.0   |  0.7814123466750394 |  3.0  | 0.23663310164846152 |    2.0    |  0.6575757575747051  |
|         SVM          |  6.0   | 0.35392805625811224 |  6.0  |  0.7608767886870238 |    6.0    | -0.5757575757554767  |
|    NEURAL NETWORK    |  5.0   |  0.6623382490628669 |  5.0  |  0.2658241937561817 |    5.0

In [8]:
ans=mcdm.rank(x_matrix, alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1],n_method="Linear2", w_vector=weight, s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
| GAUSSIAN NAIVE BAYES | 0.899858737722983  |
| LOGISTIC REGRESSION  | 0.8077066908488912 |
|    RANDOM FOREST     | 0.741494450486275  |
|    NEURAL NETWORK    | 0.6349765012245528 |
|    DECISION TREE     | 0.6171273946598121 |
|         SVM          | 0.4159048177248508 |
|     LINEAR MODEL     | 0.1159436647030548 |
+----------------------+--------------------+


In [9]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight,n_method="Linear1",s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
| GAUSSIAN NAIVE BAYES |  0.8369291071768921 |
|    RANDOM FOREST     |  0.726019066630884  |
| LOGISTIC REGRESSION  |  0.7234716485790686 |
|    DECISION TREE     |  0.6995548120150468 |
|    NEURAL NETWORK    |  0.6971091444143386 |
|         SVM          |  0.6258701315651469 |
|     LINEAR MODEL     | 0.29791608796106445 |
+----------------------+---------------------+
